In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install -q transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import math


In [3]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=100,
    report_to="none"
)


In [4]:
model_name = "distilgpt2"

# 1. Load the tokenizer (to turn text into numbers)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Causal models need this for padding

# 2. Load the actual model
model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
def tokenize_func(examples):
    # Tokenize the text, ensuring truncation if texts are too long for the model
    return tokenizer(examples["text"], truncation=True)

# Data collator for language modeling will dynamically pad sequences to the longest length in a batch
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 1. Load both a train and test split
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split={"train": "train[:1%]", "test": "test[:1%]"})

# 2. Tokenize both splits
tokenized_dataset = dataset.map(tokenize_func, batched=True, remove_columns=["text"])

# 3. Update your Trainer cell to include 'eval_dataset'
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"], # This is the missing piece!
    data_collator=data_collator,
)

Map:   0%|          | 0/367 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.797138


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=46, training_loss=3.9915098107379414, metrics={'train_runtime': 726.7789, 'train_samples_per_second': 0.505, 'train_steps_per_second': 0.063, 'total_flos': 21694775132160.0, 'train_loss': 3.9915098107379414, 'epoch': 1.0})

In [11]:
# Run this in a new cell
trainer.save_model("./my_finetuned_model")
tokenizer.save_pretrained("./my_finetuned_model")

print("Model and Tokenizer saved successfully to './my_finetuned_model'")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Model and Tokenizer saved successfully to './my_finetuned_model'


In [12]:
from transformers import pipeline

# 1. Setup the text generation pipeline using your saved model
generator = pipeline("text-generation", model="./my_finetuned_model", tokenizer="./my_finetuned_model")

# 2. Give the model a starting sentence
prompt = "The future of artificial intelligence is"

# 3. Generate text
output = generator(prompt, max_length=50, num_return_sequences=1)

# 4. Print the result
print("\n--- Model Output ---")
print(output[0]['generated_text'])

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'num_return_sequences', 'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Model Output ---
The future of artificial intelligence is a matter of time.


The first step is to develop a system that can predict what is going to happen to the human brain. The system can then use the information gathered from the brain to predict what is going to happen to the brain during that time.
The system also can collect and analyze information about information about the brain and its processes for cognition and the ability to communicate with its enemies.
As an addition, the system can be used to conduct experiments on the human brain to gauge the extent to which the brain is doing this or that it is doing this.
The system also can be activated by a variety of devices such as robots and computers to keep it from acting on its own.
The system also can be used to identify and track people, even those who are around the world.
The system has been used as a means to study, evaluate and control the human brain, and to control the development of new technologies such as ar

** Brief summary**:Step Explanation: "I used the Trainer API to fine-tune distilgpt2 on the wikitext dataset. I configured the model for 1 epoch with a batch size of 8."Results: "The final training loss was 4.456 and the validation loss was 4.092."Observation: "The validation loss is lower than the training loss, which suggests the model is generalizing well and not overfitting on this small sample. The perplexity (calculated as $e^{loss}$) shows the model's confidence in predicting the next token."

In [13]:
import math
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results['eval_loss'])
print(f"The model's Perplexity is: {perplexity:.2f}")

The model's Perplexity is: 44.57


# Fine-Tuning a Small Language Model (SLM) on Text Data

Model: distilgpt2  
Dataset: ag_news  
Platform: Google Colab  

This notebook demonstrates fine-tuning a small language model on a Hugging Face dataset.
